In [4]:
from data_preprocessing import old_load_nfl_data
from data_preprocessing import features_to_keep

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
x_df, y_df = old_load_nfl_data('nfl-play-by-play-2009-2018.csv')
X = x_df.to_numpy()
y = y_df.to_numpy()

# print(x_df.head())

# X = X[:100000]
# y = y[:100000]

C:\Users\zacha\Documents\programming\repos\Python\NFL_Play_Predictor\data_preprocessing.py:56: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  # convert the number of seconds to hours


   yardline_100  quarter_seconds_remaining  half_seconds_remaining  \
1          58.0                   0.248056                0.498056   
2          53.0                   0.237778                0.487778   
3          56.0                   0.226389                0.476389   
5          98.0                   0.221111                0.471111   
6          98.0                   0.211111                0.461111   

   game_seconds_remaining  quarter_end  drive  sp  qtr  down  ydstogo  ...  \
1                0.998056            0      1   0    1   1.0       10  ...   
2                0.987778            0      1   0    1   2.0        5  ...   
3                0.976389            0      1   0    1   3.0        8  ...   
5                0.971111            0      2   0    1   1.0       10  ...   
6                0.961111            0      2   0    1   2.0       10  ...   

   previous_play3_extra_point  previous_play3_field_goal  \
1                       False                     

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
clf = DecisionTreeClassifier()

param_grid = {
    'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], # 15
    'min_samples_split': [2, 5, 10, 15, 20, 25], # 2
    'min_samples_leaf': [1]
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best parameters:", best_params)
print("Best score:", best_score)

best_clf = grid_search.best_estimator_
train_pred = best_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)
print("Training set accuracy: ", train_accuracy)

y_pred = best_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Testing set accuracy:", test_accuracy)

Best parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best score: 0.7518698480049615
Training set accuracy:  0.8107523482283252
Testing set accuracy: 0.7559526780811661


In [7]:
feature_importances = best_clf.feature_importances_

# Get the indices of the features sorted by importance
sorted_indices = np.argsort(feature_importances)[::-1]

# Print the top n features and their importances
n_top_features = 10  # Adjust as needed
print("Top", n_top_features, "features:")
for i in range(n_top_features):
    feature_index = sorted_indices[i]
    feature_importance = feature_importances[feature_index]
    feature_name = f"Feature_{feature_index}"  # Assuming feature names are not available
    print(f"{i+1}. {feature_name}: {feature_importance}")

Top 10 features:
1. Feature_17: 0.22720027154704706
2. Feature_9: 0.14190849929198374
3. Feature_8: 0.12734438404612253
4. Feature_2: 0.12133064735882948
5. Feature_15: 0.07284710000707753
6. Feature_3: 0.053073753210976043
7. Feature_16: 0.0432724872015809
8. Feature_10: 0.03796714094055042
9. Feature_0: 0.036227791826123
10. Feature_1: 0.014793659179579286
